<a href="https://colab.research.google.com/github/silentfortin/ai-portfolio/blob/main/01-contactease-python/ContactEase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ContactEase Solutions – Contact Manager in Python

> 👨‍💻 Developed as part of the **AI Engineering Master – Week 1**

This project is a simple command-line contact manager application written in Python using Object-Oriented Programming (OOP). It allows users to add, edit, remove, view, and search contacts, and it stores contact data in a JSON file for persistence between sessions.

## 🔧 Features
- Add, edit, delete, and search contacts
- Save and load contacts from a JSON file
- Command-line interface with color-coded feedback
- Input validation and user-friendly prompts

🔗 GitHub Repository:
[📁 ai-portfolio](https://github.com/silentfortin/ai-portfolio/)


In [13]:
# Import necessary modules
import uuid
import json

from json.decoder import JSONDecodeError

In [14]:
# ANSI escape sequences for colored terminal output
RED = "\033[91m"
GREEN = "\033[92m"
YELLOW = "\033[93m"
CYAN = "\033[96m"
RESET = "\033[0m"

In [15]:
class Contact:
    '''
    Represents a contact with name, surname, phone number, and a unique ID.
    '''
    def __init__(self, name, surname, phone_number, id = None):
        # Unique identifier for the contact
        self.id = id if id else str(uuid.uuid4())
        self.name = name
        self.surname = surname
        self.phone_number = phone_number

    def __str__(self):
        '''
        String representation of the contact, formatted for terminal display.
        '''
        return f'{YELLOW}Full name: {self.name} {self.surname} | Phone: {self.phone_number}{RESET}'


class ContactsBook:
    '''
    Manages a collection of Contact objects with basic CRUD operations and file persistence.
    '''
    def __init__(self):
        self.contacts = []

    def add_contact(self, contact):
        '''
        Adds a new contact to the list and saves the updated list to file.
        '''
        self.contacts.append(contact)
        self.save_to_file()
        print(f'{GREEN}{contact.name} {contact.surname} has been added.\n')

    def show_all_contacts(self):
        '''
        Displays all contacts in the contact book.
        '''
        if self.contacts:
            print(f'\n{CYAN}Your contacts:{RESET}')
            for contact in self.contacts:
                # Uses __str__ from Contact
                print(contact)
        else:
            print(f'{YELLOW}Your contacts book is empty.\n{RESET}')

    def edit_contact(self, contact, mod, updated_var):
        '''
        Edits the specified field of a contact ('n' for name, 's' for surname, 'p' for phone).
        '''
        match mod:
            case 'n':
                contact.name = updated_var
                print(f'{GREEN}Contact name has been modified.\n')
            case 's':
                contact.surname = updated_var
                print(f'{GREEN}Contact surname has been modified.\n')
            case 'p':
                contact.phone = updated_var
                print(f'{GREEN}Contact phone number has been modified.\n')
            case _:
                print(f'{RED}ERROR: {contact.name} {contact.surname} has not been modified.\n')
                return
        self.save_to_file()

    def remove_contact(self, contact_to_remove):
        '''
        Removes a contact by comparing its unique ID.
        '''
        for contact in self.contacts:
            if contact.id == contact_to_remove.id:
                self.contacts.remove(contact)
                self.save_to_file()
                print(f'{YELLOW}Contact {contact.name} {contact.surname} removed.{RESET}')
                return
        print('Contact not found.')

    def search_contact(self, name, surname):
        '''
        Searches for a contact by matching name and surname (case-insensitive).
        '''
        name = name.strip().lower()
        surname = surname.strip().lower()

        for contact in self.contacts:
            if contact.name.lower() == name and contact.surname.lower() == surname:
                return contact
        return None

    def save_to_file(self, filename='contacts_book.json'):
        '''
        Saves the list of contacts to a JSON file.
        '''
        try:
            with open(filename, 'w') as f:
                # Convert Contact objects to dictionaries
                data = [vars(c) for c in self.contacts]
                json.dump(data, f, indent=2)
        except JSONDecodeError as e:
            print(f'Error: Failed to encode contacts to JSON. Details: {e}')
        except Exception as e:
            print(f'Unexpected error while saving contacts: {e}')

    def load_from_file(self, filename='contacts_book.json'):
        '''
        Loads contacts from a JSON file, recreating Contact objects.
        '''
        try:
            with open(filename, 'r') as file:
                data = json.load(file)
                self.contacts = [Contact(**c) for c in data]
        except FileNotFoundError:
            print(f'Warning: File "{filename}" not found. Starting with an empty contact list.')
            self.contacts = []
        except JSONDecodeError as e:
            print(f'Warning: File contains invalid JSON. Details: {e}')
            self.contacts = []
        except Exception as e:
            print(f'Unexpected error while loading contacts: {e}')
            self.contacts = []


In [16]:
# Helper function(s)

def is_valid_phone(phone):
    '''
    Validates that the phone number contains only digits and has a length between 3 and 15.
    '''
    return phone.isdigit() and 3 <= len(phone) <= 15


def prompt_valid_phone():
    '''
    Repeatedly prompts the user to enter a valid phone number until the input is valid.
    Returns the validated phone number.
    '''
    while True:
        phone = input('Insert phone number: ').strip()
        if is_valid_phone(phone):
            return phone
        print(f'{YELLOW}Invalid phone number. Only digits, min 3, max 15.{RESET}')


def prompt_non_empty_input(field_name):
    """
    Prompt the user to input a non-empty string for a given field (e.g., name or surname).
    """
    while True:
        value = input(f'Insert contact {field_name}: ').strip()
        if value:
            return value
        print(f'{RED}{field_name.capitalize()} cannot be empty.{RESET}')


def add_contact_helper(book):
    '''
    Handles the user interaction for adding a new contact.
    Prompts for name, surname, and phone. Checks for duplicates before adding.
    '''
    name = prompt_non_empty_input('name')
    surname = prompt_non_empty_input('surname')
    phone = prompt_valid_phone()

    new_contact = Contact(name, surname, phone)

    # Check for duplicate contact by name and surname
    check_duplicate = book.search_contact(new_contact.name, new_contact.surname)

    if check_duplicate is None:
        book.add_contact(new_contact)
    else:
        print(f'{YELLOW}The contact {new_contact.name} {new_contact.surname} already exists. Do you want to add it anyways?{RESET}')
        add_choice = input('Select [Y] or [X]: ').strip()
        if add_choice.lower() == 'y':
            book.add_contact(new_contact)
        else:
            print(f'{YELLOW}The contact has not been added.\n{RESET}')


def edit_contact_helper(book):
    '''
    Handles user interaction to edit a contact's name, surname, or phone.
    Prompts for the contact to edit and validates the chosen modification.
    '''
    possible_choices = ['n', 'p', 's']

    print(f'{YELLOW}Insert the name and the surname of the contact that you want to edit.{RESET}')
    name = prompt_non_empty_input('name')
    surname = prompt_non_empty_input('surname')

    contact_to_edit = book.search_contact(name, surname)

    if not contact_to_edit:
        print(f'{RED}Contact not found.{RESET}')
        return

    while True:
        edit_choice = input('\nPress [N] to edit the name\nPress [S] to edit the surname\nPress [P] to edit the phone number: ').strip().lower()
        if edit_choice in possible_choices:
            break
        else:
            print(f'{YELLOW}Invalid value. Please enter N, S, or P.{RESET}')

    if edit_choice == 'p':
        new_value = prompt_valid_phone()
    else:
        new_value = prompt_non_empty_input('new value')

    book.edit_contact(contact_to_edit, edit_choice, new_value)


def remove_contact_helper(book):
    '''
    Handles the user interaction for removing a contact by name and surname.
    '''
    name = prompt_non_empty_input('name')
    surname = prompt_non_empty_input('surname')

    contact_to_remove = book.search_contact(name, surname)
    if not contact_to_remove:
        print(f'{RED}Contact not found.{RESET}')
        return

    book.remove_contact(contact_to_remove)


def search_contact_helper(book):
    '''
    Searches for a contact by name and surname and prints the result.
    '''
    name = prompt_non_empty_input('name')
    surname = prompt_non_empty_input('surname')


    contact = book.search_contact(name, surname)
    if contact:
        print(contact)
    else:
        print(f'{RED}Contact not found.{RESET}')

### CLI

In [ ]:
book = ContactsBook()
# Load existing contacts or initialize empty list if file not found
book.load_from_file()


def main_menu():
    '''
    Displays the main CLI menu and routes the user's choice
    to the appropriate contact management function.
    '''
    while True:
        print(f'\n{CYAN}=== Contact Book ==={RESET}')
        print(f'{GREEN}1. Add contact{RESET}')
        print(f'{GREEN}2. Show contacts{RESET}')
        print(f'{GREEN}3. Modify contact{RESET}')
        print(f'{GREEN}4. Remove contact{RESET}')
        print(f'{GREEN}5. Search contact{RESET}')
        print(f'{RED}6. Exit{RESET}')

        choice = input('Choose an option (1–6): ').strip()

        if choice == '1':
            add_contact_helper(book)
        elif choice == '2':
            book.show_all_contacts()
        elif choice == '3':
            edit_contact_helper(book)
        elif choice == '4':
            remove_contact_helper(book)
        elif choice == '5':
            search_contact_helper(book)
        elif choice == '6':
            print(f'{RED}Exiting...{RESET}')
            break
        else:
            print(f'{RED}Invalid choice. Try again.{RESET}')


if __name__ == '__main__':
    main_menu()


---

## 📚 References & Resources

- [Try and Except in Python](https://pythonbasics.org/try-except/)

- [Python User Input: Handling, Validation, and Best Practices](https://www.datacamp.com/tutorial/python-user-input)

- [Python Match Case Statement](https://www.geeksforgeeks.org/python/python-match-case-statement/)

- [5 Efficient Ways to Convert a Class Object to a Dictionary in Python](https://blog.finxter.com/5-efficient-ways-to-convert-a-class-object-to-a-dictionary-in-python/)

- [Dictionary Unpacking](https://pieriantraining.com/python-unpack-dictionary-a-comprehensive-guide/)

- [Working With JSON Data in Python](https://realpython.com/python-json/#write-a-json-file-with-python)

- [Stack Overflow – How do I print colored text to the terminal?](https://stackoverflow.com/questions/287871/how-do-i-print-colored-text-to-the-terminal)

- [ANSI command line colors with Python](https://python.code-maven.com/ansi-command-line-colors-with-python)

- ChatGPT – used for improving documentation clarity and polishing markdown sections


